## 6.4 複数のデータセットをマージする

In [9]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [10]:
person = pl.read_csv("../data/survey_person.csv")
site = pl.read_csv("../data/survey_site.csv")
survey = pl.read_csv("../data/survey_survey.csv")
visited = pl.read_csv("../data/survey_visited.csv")

display(person)
display(site)
display(survey)
display(visited)

ident,personal,family
str,str,str
"""dyer""","""William""","""Dyer"""
"""pb""","""Frank""","""Pabodie"""
"""lake""","""Anderson""","""Lake"""
"""roe""","""Valentina""","""Roerich"""
"""danforth""","""Frank""","""Danforth"""


name,lat,long
str,f64,f64
"""DR-1""",-49.85,-128.57
"""DR-3""",-47.15,-126.72
"""MSK-4""",-48.87,-123.4


taken,person,quant,reading
i64,str,str,f64
619,"""dyer""","""rad""",9.82
619,"""dyer""","""sal""",0.13
622,"""dyer""","""rad""",7.8
622,"""dyer""","""sal""",0.09
734,"""pb""","""rad""",8.41
734,"""lake""","""sal""",0.05
734,"""pb""","""temp""",-21.5
735,"""pb""","""rad""",7.22
735,null,"""sal""",0.06


ident,site,dated
i64,str,str
619,"""DR-1""","""1927-02-08"""
622,"""DR-1""","""1927-02-10"""
734,"""DR-3""","""1939-01-07"""
735,"""DR-3""","""1930-01-12"""
751,"""DR-3""","""1930-02-26"""
752,"""DR-3""",null
837,"""MSK-4""","""1932-01-14"""
844,"""DR-1""","""1932-03-22"""


### 多対1でマージする

In [11]:
# 列名が異なるDataFrameを結合する場合
o2o_merge = site.join(other = visited, left_on = "name", right_on = "site")
o2o_merge

name,lat,long,ident,dated
str,f64,f64,i64,str
"""DR-1""",-49.85,-128.57,619,"""1927-02-08"""
"""DR-1""",-49.85,-128.57,622,"""1927-02-10"""
"""DR-3""",-47.15,-126.72,734,"""1939-01-07"""
"""DR-3""",-47.15,-126.72,735,"""1930-01-12"""
"""DR-3""",-47.15,-126.72,751,"""1930-02-26"""
"""DR-3""",-47.15,-126.72,752,null
"""MSK-4""",-48.87,-123.4,837,"""1932-01-14"""
"""DR-1""",-49.85,-128.57,844,"""1932-03-22"""


### 多対多でマージする

In [12]:
ps = person.join(other = survey, left_on = "ident", right_on = "person")
vs = visited.join(other = survey, left_on = "ident", right_on = "taken")

display(ps)
display(vs)

ident,personal,family,taken,quant,reading
str,str,str,i64,str,f64
"""dyer""","""William""","""Dyer""",619,"""rad""",9.82
"""dyer""","""William""","""Dyer""",619,"""sal""",0.13
"""dyer""","""William""","""Dyer""",622,"""rad""",7.8
"""dyer""","""William""","""Dyer""",622,"""sal""",0.09
"""pb""","""Frank""","""Pabodie""",734,"""rad""",8.41
"""lake""","""Anderson""","""Lake""",734,"""sal""",0.05
"""pb""","""Frank""","""Pabodie""",734,"""temp""",-21.5
"""pb""","""Frank""","""Pabodie""",735,"""rad""",7.22
"""pb""","""Frank""","""Pabodie""",751,"""rad""",4.35


ident,site,dated,person,quant,reading
i64,str,str,str,str,f64
619,"""DR-1""","""1927-02-08""","""dyer""","""rad""",9.82
619,"""DR-1""","""1927-02-08""","""dyer""","""sal""",0.13
622,"""DR-1""","""1927-02-10""","""dyer""","""rad""",7.8
622,"""DR-1""","""1927-02-10""","""dyer""","""sal""",0.09
734,"""DR-3""","""1939-01-07""","""pb""","""rad""",8.41
734,"""DR-3""","""1939-01-07""","""lake""","""sal""",0.05
734,"""DR-3""","""1939-01-07""","""pb""","""temp""",-21.5
735,"""DR-3""","""1930-01-12""","""pb""","""rad""",7.22
735,"""DR-3""","""1930-01-12""",null,"""sal""",0.06


In [13]:
ps_vs = ps.join(other = vs, left_on = ["quant"], right_on = ["quant"])
ps_vs

ident,personal,family,taken,quant,reading,ident_right,site,dated,person,reading_right
str,str,str,i64,str,f64,i64,str,str,str,f64
"""dyer""","""William""","""Dyer""",619,"""rad""",9.82,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""dyer""","""William""","""Dyer""",622,"""rad""",7.8,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""pb""","""Frank""","""Pabodie""",734,"""rad""",8.41,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""pb""","""Frank""","""Pabodie""",735,"""rad""",7.22,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""pb""","""Frank""","""Pabodie""",751,"""rad""",4.35,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""lake""","""Anderson""","""Lake""",752,"""rad""",2.19,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""lake""","""Anderson""","""Lake""",837,"""rad""",1.46,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""roe""","""Valentina""","""Roerich""",844,"""rad""",11.25,619,"""DR-1""","""1927-02-08""","""dyer""",9.82
"""dyer""","""William""","""Dyer""",619,"""sal""",0.13,619,"""DR-1""","""1927-02-08""","""dyer""",0.13
